In [1]:
import json
import requests

In [2]:
with open('nodes.json') as f:
    nodes = json.load(f)

In [3]:
class OM2M:
    def __init__(self, username, password, url):
        self.username = username
        self.password = password
        self.url = url
        
    def create_AE(self, name, labels=[], rr=False):
        data = {
            "m2m:ae": {
                "rn": name,
                "lbl": labels,
                "rr": rr,
                "api": name
            }
        }
        headers = {
            'X-M2M-Origin': self.username + ':' + self.password,
            'Content-Type': 'application/json;ty=2'
        }
        r = requests.post(self.url, headers=headers, json=data)
        return r.status_code, r.text
    
    def create_container(self, name, parent, labels=[], mni=120):
        # Create Node
        data = {
            "m2m:cnt":{
                "rn": name,
                "lbl": labels,
                "mni": mni
            }
        }
        headers = {
            'X-M2M-Origin': self.username + ':' + self.password,
            'Content-Type': 'application/json;ty=3'
        }

        r = requests.post(self.url + '/' + parent, headers=headers, json=data)

        # Create Data Container inside Node
        data = {
            "m2m:cnt":{
                "rn": "Data",
                "lbl": labels,
                "mni": mni
            }
        }

        r = requests.post(self.url + '/' + parent + '/' + name, headers=headers, json=data)

        return r.status_code

In [4]:
class Mobius:
    def __init__(self, XM2MRI, url):
        self.XM2MRI = XM2MRI
        self.url = url
    
    def create_AE(self, name, XM2MORIGIN, labels=[], rr=False):
        data = {
            "m2m:ae": {
                "rn": name,
                "api": "0.2.481.2.0001.001.000111",
                "lbl": labels,
                "rr": rr
            }
        }
        headers = {
            'X-M2M-RI': self.XM2MRI,
            'X-M2M-Origin': XM2MORIGIN,
            'Content-Type': 'application/json;ty=2'
        }

        r = requests.post(self.url, headers=headers, json=data)
        return r.status_code, r.text
    
    def create_container(self, name, parent, XM2MORIGIN, labels=[], mni=120):
        # Create Node
        data = {
            "m2m:cnt":{
                "rn": name,
                "lbl": labels,
                "mni": mni
            }
        }
        headers = {
            'X-M2M-RI': self.XM2MRI,
            'X-M2M-Origin': XM2MORIGIN,
            'Content-Type': 'application/json;ty=3'
        }

        r = requests.post(self.url + '/' + parent, headers=headers, json=data)

        # Create Data Container inside Node
        data = {
            "m2m:cnt":{
                "rn": "Data",
                "lbl": labels,
                "mni": mni
            }
        }

        r = requests.post(self.url + '/' + parent + '/' + name, headers=headers, json=data)

        return r.status_code

In [29]:
import json 

class ACME:
    def __init__(self, url, rifile, rvi):
        self.url = url
        self.rvi = rvi
        # if file exists read else empty dict
        try:
            with open(rifile, 'r') as f:
                self.ri = json.load(f)
        except:
            self.ri = {}
    
    def create_AE(self, AE, XM2MRI, labels=[], rr=False):
        data = {
            "m2m:ae": {
                "api": "N" + AE,
                "rn": AE,
                "srv": [
                    self.rvi
                ],
                "rr": rr
            }
        }

        headers = {
            'Content-Type': 'application/json;ty=2',
            'Accept': 'application/json',
            'X-M2M-Origin': 'CAdmin' + AE,
            'X-M2M-RI': XM2MRI,
            'X-M2M-RVI': self.rvi
        }

        r = requests.post(self.url + '/in-cse', headers=headers, data = json.dumps(data))
        if r.status_code == 200 or r.status_code == 201 or r.status_code == 409:
            if self.ri.get(AE) is None:
                self.ri[AE] = {}
                self.ri[AE]['ri'] = r.json()['m2m:ae']['ri']
                self.ri[AE]['nodes'] = {}
                with open('ri.json', 'w') as f:
                    json.dump(self.ri, f)
        
        return r.status_code, r.text.encode('utf8')

    def create_container(self, name, parent, XM2MRI):
            data = {
                "m2m:cnt": {
                    "rn": name,
                    "mbs": 10000,
                    "mni": 10,
                }
            }
            headers = {
                'Content-Type': 'application/json;ty=3',
                'Accept': 'application/json',
                'X-M2M-Origin': 'CAdmin' + parent,
                'X-M2M-RI': XM2MRI,
                'X-M2M-RVI': self.rvi
            }

            r = requests.post(self.url + '/' + self.ri[parent]['ri'], headers=headers, data = json.dumps(data))
            if r.status_code == 200 or r.status_code == 201 or r.status_code == 409:
                if self.ri[parent]['nodes'].get(name) is None:
                    self.ri[parent]['nodes'][name] = {}
                    self.ri[parent]['nodes'][name]['ri'] = r.json()['m2m:cnt']['ri']
                    self.ri[parent]['nodes'][name]['nodes'] = {}
                    with open('ri.json', 'w') as f:
                        json.dump(self.ri, f)

            # create Data container
            data = {
                "m2m:cnt": {
                    "rn": "Data",
                    "mbs": 10000,
                    "mni": 10,
                }
            }

            r = requests.post(self.url + '/' + self.ri[parent]['nodes'][name]['ri'], headers=headers, data = json.dumps(data))

            if r.status_code == 200 or r.status_code == 201 or r.status_code == 409:
                if self.ri[parent]['nodes'][name]['nodes'].get('Data') is None:
                    self.ri[parent]['nodes'][name]['nodes']['Data'] = {}
                    self.ri[parent]['nodes'][name]['nodes']['Data']['ri'] = r.json()['m2m:cnt']['ri']
                    with open('ri.json', 'w') as f:
                        json.dump(self.ri, f)
            
            return r.status_code

In [30]:
om2m = OM2M('admin', 'admin', 'http://10.3.1.117:8200/~/in-cse/in-name')

for node in nodes:
    print ("Creating node: " + node)
    r = om2m.create_AE(node)
    if r[0] != 200 and r[0] != 409 and r[0] != 201:
        print ("Error creating node: " + node)
        print (r)
        continue        
    for sensor in nodes[node]:
        print ("Creating sensor: " + sensor, end='\r')
        r = om2m.create_container(sensor, node)
        if r != 200 and r != 409 and r != 201:
            print ("Error creating sensor: " + sensor)
            print (r)
            exit(1)
        print ("Created sensor: " + sensor + ", response: " + str(r))

Creating node: AE-AQ
Created sensor: AQ-MG00-00, response: 409
Created sensor: AQ-BN00-00, response: 409
Created sensor: AQ-KN00-00, response: 409
Created sensor: AQ-SN00-00, response: 409
Created sensor: AQ-VN90-00, response: 409
Created sensor: AQ-KH00-00, response: 409
Created sensor: AQ-AN00-00, response: 409
Created sensor: AQ-PH03-00, response: 409
Created sensor: AQ-PL00-00, response: 409
Created sensor: AQ-FG00-00, response: 409
Created sensor: AQ-AD95-00, response: 409
Created sensor: AQ-MG00-00, response: 409
Created sensor: AQ-BN00-00, response: 409
Created sensor: AQ-KN00-00, response: 409
Created sensor: AQ-SN00-00, response: 409
Created sensor: AQ-VN90-00, response: 409
Created sensor: AQ-KH00-00, response: 409
Created sensor: AQ-AN00-00, response: 409
Created sensor: AQ-PH03-00, response: 409
Created sensor: AQ-PL00-00, response: 409
Created sensor: AQ-FG00-00, response: 409
Created sensor: AQ-AD95-00, response: 409
Creating node: AE-CM


KeyboardInterrupt: 

In [5]:
mobius = Mobius('12345', 'http://10.3.1.117:8001/Mobius')
# mobius.create_AE('test', 'S123', ['Mobius'], True)
# mobius.create_container('testnode', 'test', 'S123', ['Mobius'], 120)

for node in nodes:
    print ("Creating node: " + node)
    origin = 'SOrigin' + node
    r = mobius.create_AE(node, origin)
    if r[0] != 200 and r[0] != 409 and r[0] != 201:
        print ("Error creating node: " + node)
        print (r)
        continue        
    for sensor in nodes[node]:
        print ("Creating sensor: " + sensor, end='\r')
        r = mobius.create_container(sensor, node, origin)
        if r != 200 and r != 409 and r != 201:
            print ("Error creating sensor: " + sensor)
            print (r)
            exit(1)
        print ("Created sensor: " + sensor + ", response: " + str(r))

Creating node: AE-AQ
Created sensor: AQ-MG00-00, response: 409
Created sensor: AQ-BN00-00, response: 409
Created sensor: AQ-KN00-00, response: 409
Created sensor: AQ-SN00-00, response: 409
Created sensor: AQ-VN90-00, response: 409
Created sensor: AQ-KH00-00, response: 409
Created sensor: AQ-AN00-00, response: 409
Created sensor: AQ-PH03-00, response: 409
Created sensor: AQ-PL00-00, response: 409
Created sensor: AQ-FG00-00, response: 409
Created sensor: AQ-AD95-00, response: 409
Created sensor: AQ-MG00-00, response: 409
Created sensor: AQ-BN00-00, response: 409
Created sensor: AQ-KN00-00, response: 409
Created sensor: AQ-SN00-00, response: 409
Created sensor: AQ-VN90-00, response: 409
Created sensor: AQ-KH00-00, response: 409
Created sensor: AQ-AN00-00, response: 409
Created sensor: AQ-PH03-00, response: 409
Created sensor: AQ-PL00-00, response: 409
Created sensor: AQ-FG00-00, response: 409
Created sensor: AQ-AD95-00, response: 409
Creating node: AE-CM
Created sensor: CM-MG00-00, respon

In [33]:
acme = ACME('http://10.3.1.117:8002', 'ri.json', '3')
for node in nodes:
    print ("Creating node: " + node)
    r = acme.create_AE(node, '123')
    if r[0] != 200 and r[0] != 409 and r[0] != 201 and r[0] != 403:
        print ("Error creating node: " + node)
        print (r)
        continue        
    for sensor in nodes[node]:
        print ("Creating sensor: " + sensor, end='\r')
        r = acme.create_container(sensor, node, '123')
        if r != 200 and r != 409 and r != 201 and r != 403:
            print ("Error creating sensor: " + sensor)
            print (r)
            exit(1)
        print ("Created sensor: " + sensor + ", response: " + str(r))

Creating node: AE-AQ
Created sensor: AQ-MG00-00, response: 201
Created sensor: AQ-BN00-00, response: 201
Created sensor: AQ-KN00-00, response: 201
Created sensor: AQ-SN00-00, response: 201
Created sensor: AQ-VN90-00, response: 201
Created sensor: AQ-KH00-00, response: 201
Created sensor: AQ-AN00-00, response: 201
Created sensor: AQ-PH03-00, response: 201
Created sensor: AQ-PL00-00, response: 201
Created sensor: AQ-FG00-00, response: 201
Created sensor: AQ-AD95-00, response: 201
Created sensor: AQ-MG00-00, response: 409
Created sensor: AQ-BN00-00, response: 409
Created sensor: AQ-KN00-00, response: 409
Created sensor: AQ-SN00-00, response: 409
Created sensor: AQ-VN90-00, response: 409
Created sensor: AQ-KH00-00, response: 409
Created sensor: AQ-AN00-00, response: 409
Created sensor: AQ-PH03-00, response: 409
Created sensor: AQ-PL00-00, response: 409
Created sensor: AQ-FG00-00, response: 409
Created sensor: AQ-AD95-00, response: 409
Creating node: AE-CM
Created sensor: CM-MG00-00, respon